In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
np.random.seed(1)

np.set_printoptions(threshold=np.inf)

brain_size = 12


class Maze(object):
    def __init__(self, length, width): 
        self.length = length
        self.width = width
        self.maze = np.ones((self.width, self.length)) *2

    def clear_maze(self):
        self.maze = np.ones((self.width, self.length)) *2
    
    def generate_maze(self):
        #wall_position = np.random.randint(2, size=self.length)
        self.wall_position = np.arange(1,self.length,2)
        self.road_position = np.arange(0,self.length,2)
        #print(self.wall_position)
        
        self.door_position = np.random.randint(low=1, high=self.width-1, size=len(self.wall_position))
        #print(self.door_position)
    
        for i in self.road_position:
            self.maze[1:-1,i]=8
            
        
        self.maze[self.door_position[-1], self.wall_position[-1]] = 1
        for i in np.arange(len(self.wall_position)-2, -1, -1):
            if self.door_position[i+1] < self.door_position[i]:
                self.maze[self.door_position[i], self.wall_position[i]] = 0
            else: 
                self.maze[self.door_position[i], self.wall_position[i]] = 1
                
    def print_maze(self, x=-1, y=-1):
        if x>=0 and y>=0:
            tmp = self.maze[x,y]
            self.maze[x,y] = -1 # position of the agent
            
        print("  ", end="")    
        for i in np.arange(self.length):
            print('%d ' % i, end='')
        print("\n")
        
        for j in np.arange(self.width):
            print('%d ' % j, end='')
            for i in np.arange(self.length):
            
                if self.maze[j,i]==2: # wall position
                    print('x ',end='')
                elif self.maze[j,i]==8:
                    print('o ',end='')# road
                elif self.maze[j,i]==-1:
                    print('T ',end='')
                    self.maze[x,y]= tmp
                else:
                    print('%d ' % self.maze[j,i], end='')
            print('\n')
        
        
    
    
class Agent_circular:
    def __init__(self, maze, num_gene=4, brain_size=12, genome_len = 2000, probabilistic=True, gene=None):
        
        self.maze = maze
        self.brain_size = brain_size
        self.brain = np.zeros(self.brain_size)
        
        self.num_gene = num_gene
        self.gene_mask = np.zeros((self.num_gene,2)) # the input output size of each gate
        self.max_markov_gate_inputs = 4
        self.max_markov_gate_outputs = 3
        
        
        self.markov_gates = []
        self.markov_gate_input_ids = []
        self.markov_gate_output_ids = []
        #self.gene = np.array([Gate(self.gene_mask[i,0], self.gene_mask[i,1]) for i in range(self.num_gene)])
        if gene is None:
            self.gene = np.random.randint(0, 256, genome_len).astype(np.uint8)
            
            # Seed the random genome with seed_num_markov_gates Markov Gates
            for _ in range(self.num_gene):
                start_index = np.random.randint(0, int(len(self.gene) * 0.8))
                self.gene[start_index] = 42
                self.gene[start_index + 1] = 213
            """for index_counter in np.arange(self.gene.shape[0] - 1):
            # Sequence of 42 then 213 indicates a new Markov Gate
                if self.gene[index_counter] == 42 and self.gene[index_counter + 1] == 213:
                    self.gene[index_counter] = 85
                    self.gene[index_counter+1] = 211
                    
            """
        else:
            self.gene = np.array(gene, dtype=np.uint8)

        self._setup_markov_network(probabilistic)
        
        
        self.end = False # reach the end of maze
        self.time_step = 0 # +1 for every move
        self.thinking_times = 0 # +1 for every step
        #self.life = np.maximum(300, 10*self.maze.length)
        self.life = 300
        self.pass_maze = 0
        
        self.position = np.array([np.random.choice(np.arange(1,self.maze.width-1)), 0])
        self.trajectory = np.ones((self.life, 2))*-1
        self.trajectory[self.time_step,:] = self.position
        
        self.door_direction()
        
        self.perception()
        
    def door_direction(self):
        left = self.maze.maze[1:self.position[0], 1]
        right = self.maze.maze[self.position[0]:self.maze.width-1, 1]
        for land in left:
            if land != 2: 
                self.brain[3] = 0
                break
        for land in right:
            if land != 2: 
                self.brain[3] = 1
                break
        
     # reinit agent after mutation, regenerate markov gates
    def reinit(self):
        self.brain = np.zeros(self.brain_size)
        
        if self.markov_gates == []:
            # Seed the random genome with seed_num_markov_gates Markov Gates
            for _ in range(1):
                start_index = np.random.randint(0, int(len(self.gene) * 0.8))
                self.gene[start_index] = 42
                self.gene[start_index + 1] = 213
        
        self.markov_gates = []
        self.markov_gate_input_ids = []
        self.markov_gate_output_ids = []

        
        self._setup_markov_network(probabilistic=True)
        
        
        self.end = False # reach the end of maze
        self.time_step = 0 # +1 for every move
        self.thinking_times = 0 # +1 for every step
        #self.life = np.maximum(300, 10*self.maze.length)
        self.life = 300
        self.pass_maze = 0
        
        self.position = np.array([np.random.choice(np.arange(1,self.maze.width-1)), 0])
        self.trajectory = np.ones((self.life, 2))*-1
        self.trajectory[self.time_step,:] = self.position
        
        self.door_direction()
        
        self.perception()
        
    # reinit when the genome has no changes, used in fitness evaluation
    def simple_reinit(self):
        
        self.brain = np.zeros(self.brain_size)
 
        self.end = False # reach the end of maze
        self.time_step = 0 # +1 for every move
        self.thinking_times = 0 # +1 for every step
        #self.life = np.maximum(300, 10*self.maze.length)
        self.life = 300
        self.pass_maze = 0
        
        self.position = np.array([np.random.choice(np.arange(1,self.maze.width-1)), 0])
        self.trajectory = np.ones((self.life, 2))*-1
        self.trajectory[self.time_step,:] = self.position
        
        self.door_direction()
        self.perception()
    
    def _setup_markov_network(self, probabilistic):
        """Interprets the internal genome into the corresponding Markov Gates

        Parameters
        ----------
        probabilistic: bool
            Flag indicating whether the Markov Gates are probabilistic or deterministic

        Returns
        -------
        None

        """
        num_gene = 0
        tmp = []
        for index_counter in np.arange(self.gene.shape[0] - 1):
            # Sequence of 42 then 213 indicates a new Markov Gate
            if self.gene[index_counter] == 42 and self.gene[index_counter + 1] == 213:
                internal_index_counter = (index_counter + 2)%self.gene.shape[0]
                num_gene += 1

                # Determine the number of inputs and outputs for the Markov Gate
                num_inputs = (self.gene[internal_index_counter] % self.max_markov_gate_inputs) + 1
                internal_index_counter += 1 
                internal_index_counter = internal_index_counter%self.gene.shape[0]
                num_outputs = (self.gene[internal_index_counter] % self.max_markov_gate_outputs) + 1
                internal_index_counter += 1
                internal_index_counter = internal_index_counter%self.gene.shape[0]
                
                tmp.append([num_inputs, num_outputs]) 
                # Make sure that the genome is long enough to encode this Markov Gate
                if (internal_index_counter +
                        (self.max_markov_gate_inputs + self.max_markov_gate_outputs) +
                        (2 ** num_inputs) * (2 ** num_outputs)) > self.gene.shape[0]:
                    continue

                # Determine the states that the Markov Gate will connect its inputs and outputs to
                input_state_ids = self.gene[internal_index_counter:internal_index_counter + self.max_markov_gate_inputs][:num_inputs]
                input_state_ids = np.mod(input_state_ids, self.brain.shape[0])
                #input_state_ids = np.rint(np.array(input_state_ids)*12/255-0.5).astype(np.int32)
                internal_index_counter += self.max_markov_gate_inputs
                internal_index_counter = internal_index_counter%self.gene.shape[0]

                output_state_ids = self.gene[internal_index_counter:internal_index_counter + self.max_markov_gate_outputs][:num_outputs]
                output_state_ids = np.mod(output_state_ids, 6)+6
                #output_state_ids = np.rint(np.array(output_state_ids*6/255+5.5)).astype(np.int32)
                internal_index_counter += self.max_markov_gate_outputs
                internal_index_counter = internal_index_counter%self.gene.shape[0]

                self.markov_gate_input_ids.append(input_state_ids)
                self.markov_gate_output_ids.append(output_state_ids)

                # Interpret the probability table for the Markov Gate
                markov_gate = np.copy(self.gene[internal_index_counter:internal_index_counter + (2 ** num_inputs) * (2 ** num_outputs)])
                markov_gate = markov_gate.reshape((2 ** num_inputs, 2 ** num_outputs))

                if probabilistic:  # Probabilistic Markov Gates
                    markov_gate = markov_gate.astype(np.float64) / np.sum(markov_gate, axis=1, dtype=np.float64)[:, None]
                    # Precompute the cumulative sums for the activation function
                    # markov_gate = np.cumsum(markov_gate, axis=1, dtype=np.float64)
                else:  # Deterministic Markov Gates
                    row_max_indices = np.argmax(markov_gate, axis=1)
                    markov_gate[:, :] = 0
                    markov_gate[np.arange(len(row_max_indices)), row_max_indices] = 1

                self.markov_gates.append(markov_gate)
        self.num_gene = num_gene
        self.gene_mask = np.array(tmp)

        
    def brain_update(self, times=1):

        x,y = self.position
        if y == self.maze.length-1: # reach the end of the maze
            self.pass_maze = self.pass_maze + 1
            self.init_locate()
        self.perception()
        
        # Save original input values
        original_input_values = np.copy(self.brain[:6])
        
        for _ in range(times):
           
            all_outputs_idx = np.array([])
            for gate_output in self.markov_gate_output_ids:
                all_outputs_idx = np.concatenate((all_outputs_idx, gate_output))
            all_outputs_idx = np.unique(all_outputs_idx).astype(int)
            
            self.brain[all_outputs_idx] = 0
            # NOTE: This routine can be refactored to use NumPy if larger MNs are being used
            # See implementation at https://github.com/rhiever/MarkovNetwork/blob/a381aa9919bb6898b56f678e08127ba6e0eef98f/MarkovNetwork/MarkovNetwork.py#L162:L169
            for markov_gate, mg_input_ids, mg_output_ids in zip(self.markov_gates, self.markov_gate_input_ids,
                                                                self.markov_gate_output_ids):

                mg_input_index, marker = 0, 1
                # Create an integer from bytes representation (loop is faster than previous implementation)
                for mg_input_id in reversed(mg_input_ids):
                    if self.brain[mg_input_id]:
                        mg_input_index += marker
                    marker *= 2

                # Determine the corresponding output values for this Markov Gate
                #roll = np.random.uniform()  # sets a roll value
                markov_gate_subarray = markov_gate[mg_input_index,:]  # selects a Markov Gate subarray

                # Searches for the first value where markov_gate > roll
                #for i, markov_gate_element in enumerate(markov_gate_subarray):
                #    if markov_gate_element >= roll:
                #        mg_output_index = i
                #        break
                mg_output_index = np.random.choice(len(markov_gate_subarray),p = markov_gate_subarray)
                # Converts the index into a string of '1's and '0's (binary representation)
                mg_output_values = bin(mg_output_index)  # bin() is much faster than np.binaryrepr()

                # diff_len deals with the lack of the width argument there was on np.binaryrepr()
                """diff_len = mg_output_ids.shape[0] - (len(mg_output_values) - 2)
                print("\n")
                print(mg_output_ids.shape)
                print("\n")
                """
                # Loops through 'mg_output_values' and alter 'self.states'
                for i, mg_output_value in enumerate(mg_output_values[2:]):
                    if mg_output_value == '1':
                        self.brain[mg_output_ids[i]] = 1   #.astype(np.int32)
                
                    
            # Replace original input values
            self.brain[:6] = original_input_values

        
    def copy(self, agent):
        self.brain_size = agent.brain_size
        self.maze = agent.maze
        self.brain = agent.brain
        
        self.markov_gates = agent.markov_gates
        self.markov_gate_input_ids = agent.markov_gate_input_ids
        self.markov_gate_output_ids = agent.markov_gate_output_ids
        
        
        self.num_gene = agent.num_gene
        self.gene_mask = agent.gene_mask # the input output size of each gate
        self.gene = agent.gene
        
        self.end = agent.end # reach the end of maze
        self.time_step = agent.time_step
        self.thinking_times = agent.thinking_times
        self.life = agent.life
        self.pass_maze = agent.pass_maze
        
        self.position = agent.position
        self.trajectory = agent.trajectory
        
        self.perception()
        
    def init_locate(self):
        self.position = np.array([np.random.choice(np.arange(1,self.maze.width-1)), 0])
        self.end = False
        self.door_direction()
        
    def init_brain(self):
        self.brain = np.zeros(self.brain_size)
        self.time_step = 0
        self.thinking_times = 0
        self.trajectory = np.ones((self.life, 2))* -1
        
    def forget_past(self):
        self.brain[0:3] = 0
        self.brain[4:6] =0
        
    def perception(self):
        x,y = self.position
        self.forget_past()
        
        if self.maze.maze[x,y+1] == 2:
            self.brain[0]=1
        else: self.brain[0]=0
        
        if self.maze.maze[x-1,y+1] == 2:
            self.brain[1]=1
        else: self.brain[1]=0
        
        if self.maze.maze[x+1,y+1] == 2:
            self.brain[2] = 1
        else: self.brain[2]=0
        
        if self.maze.maze[x-1,y] == 2:
            self.brain[4]=1
        else: self.brain[4]=0
        
        if self.maze.maze[x+1,y] == 2:
            self.brain[5]=1
        else: self.brain[5]=0
        
        if y in self.maze.wall_position:
            self.brain[3] = self.maze.maze[x, y]
        

    
    def step(self):
        x,y = self.position
        fitness = 0
        time_step_shot = self.time_step
        self.thinking_times = self.thinking_times + 1
        # print("time_step:%d" % self.time_step)
        # print("thinking time: %d" % self.thinking_times)
        if self.thinking_times>=self.life-1:# or self.thinking_times >= 3000: 
            self.end = True
            fitness = self.get_fitness()
        
        elif y == self.maze.length-1: # reach the end of the maze
            self.pass_maze = self.pass_maze + 1
            self.init_locate()
            
        elif y in self.maze.wall_position: # in a door
            self.position = x,y+1
            self.time_step = self.time_step+1
        elif y+1 in self.maze.wall_position and self.maze.maze[x,y+1]!=2: # before a door
            #print('before a door >;<')
            self.position = x,y+1
            self.time_step = self.time_step+1
            
        elif self.brain[10] == 1 and self.brain[11] == 0:
            if self.maze.maze[x+1,y]==2:
                self.brain[10] = 0
                self.brain[11] = 1
 
            else:
                self.maze.maze[x+1,y] != 2
                self.position = x+1, y
                self.time_step = self.time_step+1
        elif self.brain[10] == 0 and self.brain[11] == 1:
            if self.maze.maze[x-1,y] == 2:
                self.brain[10] = 1
                self.brain[11] = 0
            else:
                self.maze.maze[x-1,y] != 2
                self.position = x-1, y
                self.time_step = self.time_step+1
                
        elif self.brain[10] == 1 and self.brain[11] == 1:
            if self.maze.maze[x,y+1] != 2:
                self.position = x,y+1
                self.time_step = self.time_step+1
            
        elif self.brain[10] == 0 and self.brain[11] == 0:
            self.position = x,y
            self.time_step = self.time_step+1
            '''else:
                # shouldn't have this
                self.brain[10] = 1
                self.brain[11] = 0
            '''    
        '''elif self.brain[10] == 0 and self.brain[11] == 0:
            self.brain[10] = 0
            self.brain[11] = 1
        ''' 
        
        # if the brain's order is legal, keep it
        # illegal order is omitted
        if self.time_step > time_step_shot:    
            self.trajectory[self.time_step,:] = self.position
        
        return fitness
    
    def get_fitness(self):
        x,y = self.position
        tmp1 = (y)/self.maze.length
        tmp2 = self.pass_maze
        return tmp1+tmp2
        
def test():        
    maze = Maze(10,6)
    maze.generate_maze()
    sca=maze.print_maze()

    agent = Agent_circular(maze)

    while (agent.end == False):
        #maze.print_maze(agent.position[0], agent.position[1])
        agent.brain_update(1)
        print(agent.brain)
        fitness = agent.step()
        
   
    print(fitness)
    print(agent.trajectory)
    #print(agent.gene)


#np.random.seed(9)
#test()

        
                
            
    
        

In [ ]:
import pickle
import matplotlib.pyplot as plt
np.set_printoptions(precision=2)

np.random.seed(1)


num_generation = 100
pop_size = 20

per_site_mut_rate = 0.004

per_site_inc_rate = 0.004
per_site_insert_rate = 0.025


copy_rate = 0.0001
copy_minsize = 128
copy_maxsize = 512

del_rate = 0.0001
del_minsize = 128
del_maxsize = 512

min_size = 1000
max_size = 8000

fit_times = 10





def init_pop_agent(pop_size, maze):
    pop = np.array([])
    for i in np.arange(pop_size):
        pop = np.append(pop, Agent_circular(maze))
    #    pop.append(Agent_circular(maze))
    return pop

        

def mutate(child):
    
    howmanyPoint = np.random.binomial(len(child.gene), per_site_mut_rate)
    howmanyCopy = np.random.binomial(len(child.gene), copy_rate)
    howmanyDelete = np.random.binomial(len(child.gene), del_rate)
    
    howmanyInc = np.random.binomial(len(child.gene), per_site_inc_rate)
    #print("howmanyPoint=%d,howmanyInc=%d,howmanyCopy=%d,howmanyDelete=%d" % (howmanyPoint, howmanyInc, howmanyCopy, howmanyDelete))
    
    # point mutation replacement
    mutation_point_ind = np.random.randint(low=0, high=len(child.gene), size = howmanyPoint)
    for i in np.arange(howmanyPoint):
        child.gene[mutation_point_ind[i]] = np.random.randint(0, 256, dtype=np.uint8)
    
    mutation_inc_ind = np.random.randint(low=0, high=len(child.gene), size = howmanyInc)
    for i in np.arange(howmanyInc):
        child.gene[mutation_inc_ind[i]] = (int(child.gene[mutation_inc_ind[i]])+int(np.random.randint(0, 20, dtype=np.uint8))) % 255
        
        
    # copy
    for i in np.arange(howmanyCopy):
        if len(child.gene)<max_size:
            seg_size = np.random.randint(low=0, high=copy_maxsize-copy_minsize) + copy_minsize
            if seg_size > len(child.gene): print("Error copy size")
            seg_start = np.random.randint(low=0, high=len(child.gene)-seg_size)
            segment = child.gene[seg_start:seg_start+seg_size]
            child.gene = np.insert(child.gene, np.ones(seg_size, dtype=int)*seg_start, segment)
            
    # delete
    for i in np.arange(howmanyDelete):
        if len(child.gene)>min_size:
            seg_size = np.random.randint(low=0, high=del_maxsize-del_minsize) + del_minsize
            if seg_size > len(child.gene): print("Error delete size")
            seg_start = np.random.randint(low=0, high=len(child.gene)-seg_size)
            child.gene = np.delete(child.gene, seg_start+np.arange(seg_size))
            
    return child

    
            
    
def select(fitness, k):    # nature selection wrt pop's fitness
    #idx = np.random.choice(np.arange(pop_size), size=pop_size-k, replace=True,
    #                       p=(fitness_pop+1)/(fitness_pop+1).sum())
    fitness_pop = np.copy(fitness)
    #fitness_pop[:k]=0
    count = np.zeros(len(fitness_pop))
    ret_idx = np.zeros(len(fitness_pop)-k, dtype = int)
    for i in np.arange(len(fitness_pop)-k):
        idx = np.random.choice(np.arange(pop_size), size=1, replace=True,
                           p= fitness_pop/(fitness_pop).sum()) 
        ret_idx[i] = idx
        count[idx] += 1
        if count[idx]>=2:
            fitness_pop[idx]=0
    return ret_idx



def random_search():
    # initialize the maze environment
    maze = Maze(10,6)
    maze.generate_maze()
    
    pop_size = 1000000
    batch_num = 1000
    batch_size = (int)(pop_size/batch_num)
    
    
    fitness_pop = np.zeros((batch_num, batch_size))
    fitness_gmean_pop = np.zeros((batch_num, batch_size))
    genome = []
    
    for i in np.arange(batch_num):
        if i% 10 == 0:
            print(i)
        # initialize the agent population
        pop = init_pop_agent(batch_size, maze)
        genome_pop = []
        for ind, agent_ in enumerate(pop):
            fitness_tmp = np.zeros(fit_times)
            for repeat_fit in np.arange(fit_times):
                agent_.simple_reinit()
                while(agent_.end==False):
                    agent_.brain_update(1)
                    fitness_once = agent_.step()
                fitness_tmp[repeat_fit] = fitness_once

            fitness_normal_tmp = (fitness_tmp+0.000001)/np.amax((fitness_tmp+0.000001))
            fitness = np.mean(fitness_tmp)
            fitness_gmean = pow(np.prod(fitness_normal_tmp), 1/fit_times)
            
            fitness_pop[i, ind] = fitness
            fitness_gmean_pop[i, ind] = fitness_gmean
            genome_pop.append(agent_.gene)
        genome.append(genome_pop)
            
        with open('./save_model/random_search_fitness_pop','wb') as f:
            pickle.dump(fitness_pop[:i+1,:], f)
        with open('./save_model/random_search_fitness_gmean_pop','wb') as f:
            pickle.dump(fitness_gmean_pop[:i+1,:], f)
        with open('./save_model/genome','wb') as f:
            pickle.dump(genome, f)
            
    
    return tmp,fitness_pop



def evolution():
    # initialize the maze environment
    maze = Maze(10,7)
    maze.generate_maze()

    # initialize the agent population
    pop = init_pop_agent(pop_size, maze)

    # save statistics
    generation_trend = np.zeros(num_generation)
    elite_trend = np.zeros(num_generation)
    generation_trend_meanfit = np.zeros(num_generation)
    elite_trend_meanfit = np.zeros(num_generation)
    
    generation_pop_allfit = np.zeros((num_generation, pop_size, fit_times))
    generation_pop_fit = np.zeros((num_generation, pop_size))
    
    best_genomes = np.array([])
    
    # evole
    for i_ in np.arange(num_generation):
        if i_ % 1 == 0:
            print("\n")
            print("%d iteration:" % i_)
            
            
        # eval the population fitness
        fitness_pop = np.zeros(pop_size, dtype = np.float32)
        fitness_pop_meanfit = np.zeros(pop_size, dtype = np.float32)
        
        for ind, agent_ in enumerate(pop):
            fitness_tmp = np.zeros(fit_times)
            for repeat_fit in np.arange(fit_times):
                agent_.simple_reinit()
                while(agent_.end==False):
                    agent_.brain_update(1)
                    fitness_once = agent_.step()
                fitness_tmp[repeat_fit] = fitness_once
            generation_pop_allfit[i_,ind,:] = fitness_tmp
            


#            fitness_tmp = (fitness_tmp)/30
            fitness = pow(np.prod(np.exp(fitness_tmp)), 1/fit_times)
            fitness_pop[ind] = fitness
            generation_pop_fit[i_,ind] = fitness
            if fitness>20:
                np.append(best_genomes, agent_.gene)
            
            fitness_meanfit = np.mean(fitness_tmp)
            fitness_pop_meanfit[ind] = fitness_meanfit

            
            
        generation_trend[i_] = np.mean(fitness_pop)
        generation_trend_meanfit[i_] = np.mean(fitness_pop_meanfit)
        #print(fitness_pop)

        # keep k elites without mutation
        k = 3
        elite_idx = np.argpartition(fitness_pop, -k)[-k:]
        elites = np.copy(pop[elite_idx])
        
        # select & mutation
        others_idx = np.argpartition(fitness_pop, -k)[:-k]

        sort_fit = np.concatenate((fitness_pop[elite_idx], fitness_pop[others_idx]))
        print(sort_fit)
        idx = select(sort_fit, k)
        other_pop = np.copy(pop[idx])
        for mut_ind, parent in enumerate(other_pop):
            #if np.random.rand()<0.05:
                mutate(parent)
                #print("mutate: from %d to %d)" % (idx[mut_ind], mut_ind+k))
         
        #new_idx = np.array(np.concatenate((elite_idx, idx)))
        #pop = pop[new_idx]
        pop = np.concatenate((elites, other_pop), axis=None)
        for agent in pop:
            agent.reinit()
            
        
            
            
        elite_trend[i_] = np.mean(fitness_pop[elite_idx])
        elite_trend_meanfit[i_] = np.mean(fitness_pop_meanfit[elite_idx])
    
    

    plt.title('Evolution Trend')
    plt.plot(np.arange(num_generation), np.log(generation_trend), color='green', label='Generation gmean trend',linestyle = ':')
    plt.plot(np.arange(num_generation), np.log(elite_trend), color='red', label='Elite gmean trend',linestyle='-.')
    plt.plot(np.arange(num_generation), generation_trend_meanfit, color='blue', label='Generation mean trend',linestyle = ':')
    plt.plot(np.arange(num_generation), elite_trend_meanfit, color='black', label='Elite mean trend',linestyle='-.')
    plt.xlabel("Evolution times")
    plt.ylabel("fitness (number of maze passes)")
    plt.legend() # 显示图例
    plt.show()

    
    count=0
    for agent in pop:
        if agent.markov_gates!=[]:
            count+=1
    print("useful agent: %d" % count)
    
    
    with open("./save_model/generation_trend.pickle","wb") as f:
        pickle.dump(generation_trend, f)
    with open("./save_model/elite_trend.pickle","wb") as f:
        pickle.dump(elite_trend, f)
    with open("./save_model/generation_trend_meanfit.pickle","wb") as f:
        pickle.dump(generation_trend_meanfit, f)
    with open("./save_model/elite_trend_meanfit.pickle","wb") as f:
        pickle.dump(elite_trend_meanfit, f)
    with open("./save_model/best_genomes.pickle","wb") as f:
        pickle.dump(best_genomes, f)
        
        
    with open("./save_model/generation_pop_allfit.pickle","wb") as f:
        pickle.dump(generation_pop_allfit, f)
    with open("./save_model/generation_pop_fit.pickle","wb") as f:
        pickle.dump(generation_pop_fit, f)
    


print("Running")
evolution()

print("End")



In [ ]:
with open("./save_model/generation_trend.pickle","rb") as f:
    fit = pickle.load(f)
plt.plot(np.arange(len(fit)), fit, color='green', label='Generation gmean trend',linestyle = ':')
    